# Testing the correlation between discrete traits

### Loading library

In [ ]:
library("corHMM")
library("mclust")
library("stringr")
library("phytools")
library("qpcR")
library("secsse")
library("ggtree")
library("RColorBrewer")
library("ggplot2")
library("ggpmisc")
library("TreeTools")

Le chargement a nécessité le package : ape

Le chargement a nécessité le package : nloptr

Le chargement a nécessité le package : GenSA



### Loading data

In [ ]:
df<-read.csv("Code_P_Muridae.tsv", sep="\t", header = TRUE) # omit sep ="\t" for .csv files
phy<-read.tree("Tree_65_sp_murids.tree")

In [ ]:
phy<-AddTip(phy,
  where = which(phy$tip.label== "Hyomys_goliath"),
  label = "Hyomys_goliath2",
  edgeLength = 0.1,
  lengthBelow = 0.1
)

### Cleaning and preparing data

In [ ]:
states_traits<-df[,c(1,2)]
colnames(states_traits)<-c("Species", "Cat")
states_traits[is.na(states_traits)]<-"?"

### One rate models

### Model 1 : equal rates

In [ ]:
pt_1_eq<-corHMM(phy, states_traits, rate.cat = 1, rate.mat=NULL, model = "ER", node.states = "marginal",
fixed.nodes=FALSE, p=NULL, root.p="yang", ip=NULL, nstarts=0, n.cores=5,
get.tip.states = FALSE, lewis.asc.bias = FALSE, collapse = TRUE, lower.bound = 1e-9,
upper.bound = 100, opts=NULL)

### Model 2 : all rates differ

In [ ]:
pt_1_ard<-corHMM(phy, states_traits, rate.cat = 1, rate.mat=NULL, model = "ARD", node.states = "marginal",
fixed.nodes=FALSE, p=NULL, root.p="yang", ip=NULL, nstarts=0, n.cores=5,
get.tip.states = FALSE, lewis.asc.bias = FALSE, collapse = TRUE, lower.bound = 1e-9,
upper.bound = 100, opts=NULL)

### Model 1 : equal rates

In [ ]:
pt_2_eq<-corHMM(phy, states_traits, rate.cat = 2, rate.mat=NULL, model = "ER", node.states = "marginal",
fixed.nodes=FALSE, p=NULL, root.p="yang", ip=NULL, nstarts=0, n.cores=5,
get.tip.states = FALSE, lewis.asc.bias = FALSE, collapse = TRUE, lower.bound = 1e-9,
upper.bound = 100, opts=NULL)

### Model 2 : all rates differ

In [ ]:
pt_2_ard<-corHMM(phy, states_traits, rate.cat = 2, rate.mat=NULL, model = "ARD", node.states = "marginal",
fixed.nodes=FALSE, p=NULL, root.p="yang", ip=NULL, nstarts=0, n.cores=5,
get.tip.states = FALSE, lewis.asc.bias = FALSE, collapse = TRUE, lower.bound = 1e-9,
upper.bound = 100, opts=NULL)

### Model 1 : equal rates

In [ ]:
pt_3_eq<-corHMM(phy, states_traits, rate.cat = 3, rate.mat=NULL, model = "ER", node.states = "marginal",
fixed.nodes=FALSE, p=NULL, root.p="yang", ip=NULL, nstarts=0, n.cores=5,
get.tip.states = FALSE, lewis.asc.bias = FALSE, collapse = TRUE, lower.bound = 1e-9,
upper.bound = 100, opts=NULL)

### Model 2 : all rates differ

In [ ]:
pt_3_ard<-corHMM(phy, states_traits, rate.cat = 3, rate.mat=NULL, model = "ARD", node.states = "marginal",
fixed.nodes=FALSE, p=NULL, root.p="yang", ip=NULL, nstarts=0, n.cores=5,
get.tip.states = FALSE, lewis.asc.bias = FALSE, collapse = TRUE, lower.bound = 1e-9,
upper.bound = 100, opts=NULL)

In [ ]:
df_trait<-data.frame(cbind(c(pt_1_eq$AICc,  pt_1_ard$AICc,
                          pt_2_eq$AICc, pt_2_ard$AICc,
                          pt_3_eq$AICc,  pt_3_ard$AICc),
                akaike.weights(c(pt_1_eq$AICc, pt_1_ard$AICc,
                          pt_2_eq$AICc, pt_2_ard$AICc,
                          pt_3_eq$AICc,  pt_3_ard$AICc))$deltaAIC,
                akaike.weights(c(pt_1_eq$AICc, pt_1_ard$AICc,
                          pt_2_eq$AICc, pt_2_ard$AICc, 
                          pt_3_eq$AICc, pt_3_ard$AICc))$weights,
                c(
(max(as.vector(pt_1_eq$index.mat)[!is.na(as.vector(pt_1_eq$index.mat))])), (max(as.vector(pt_1_ard$index.mat)[!is.na(as.vector(pt_1_ard$index.mat))])),
(max(as.vector(pt_2_eq$index.mat)[!is.na(as.vector(pt_2_eq$index.mat))])), (max(as.vector(pt_1_ard$index.mat)[!is.na(as.vector(pt_1_ard$index.mat))])),
(max(as.vector(pt_3_eq$index.mat)[!is.na(as.vector(pt_3_eq$index.mat))])), (max(as.vector(pt_1_ard$index.mat)[!is.na(as.vector(pt_1_ard$index.mat))])))
                ))
rownames(df_trait)<-c("pt_1_eq", "pt_1_ard",
                          "pt_2_eq",  "pt_2_ard",
                          "pt_3_eq",  "pt_3_ard")
colnames(df_trait)<-c("AICc", "Delta_AICc", "AICcWt", "K_rates")
write.table(df_trait, "Model_binary_trait.tsv", sep ="\t")
saveRDS(eval(parse(text = rownames(df_trait)[which.min(df_trait$AICc)])),paste("data_BF_PT.rds", sep =""))

In [ ]:
df_trait

In [ ]:
best_fitting_model<-readRDS("data_BF_PT.rds")

In [ ]:
df$V2<-gsub("_", " ", df$V2)

In [ ]:
for (i in 1:length(df$V1)){
    phy$tip.label[phy$tip.label==df$V1[i]]<-df$V2[i]
}

In [ ]:
states_traits[,1]<-df[,2]

In [ ]:
ASE_plot_maker<-function(phy, trait_model, data_states_model){
phylo<-ggtree(phy) +
           theme_bw() +
           theme(panel.border = element_blank(),
           legend.key = element_blank(),
           axis.ticks = element_blank(),
           axis.text.y = element_blank(),
           axis.text.x = element_blank(),
           panel.grid = element_blank(),
           panel.grid.minor = element_blank(), 
           panel.grid.major = element_blank(),
           panel.background = element_blank(),
           plot.background = element_rect(fill = "transparent",colour = NA))

node_states_traits<-trait_model$states

### assuming you have no polytomies

node_states<-c((length(phy$tip.label)+1):(2*length(phy$tip.label)-1))  

nb_states<-length(unique(trait_model$data[,2]))
node_states_traits_ase<-c()
for (i in 1:nb_states){
    if(trait_model$rate.cat == 1){
        col<-node_states_traits[,i]
        node_states_traits_ase<-cbind(node_states_traits_ase, col)
    }
        if(trait_model$rate.cat == 2){
        col<-node_states_traits[,i] + node_states_traits[,i+nb_states] 
        node_states_traits_ase<-cbind(node_states_traits_ase, col)
    }
        if(trait_model$rate.cat == 3){
        col<-node_states_traits[,i] + node_states_traits[,i+nb_states] + node_states_traits[,i+2*nb_states] 
        node_states_traits_ase<-cbind(node_states_traits_ase, col)
    }
}
    node_states_traits_ase<-cbind(node_states_traits_ase, node_states)
    node_states_traits_ase<-as.data.frame(node_states_traits_ase)
    colnames(node_states_traits_ase)<-c(1:nb_states,"node")
    saveRDS(node_states_traits, "data_categorical_trait.rds")
    pies <- nodepie(node_states_traits_ase, cols=1:nb_states, color=c("#D2B48C", "#40826D"), alpha=1)
    df<-tibble::tibble(node=as.numeric(node_states_traits_ase$node), pies=pies)
    phylo_node <- phylo %<+% df
    phylo_complete<-phylo_node + geom_plot(data=td_filter(!isTip), mapping=aes(x=x,y=y, label=pies), vp.width=0.03, vp.height=0.03, hjust=0.5, vjust=0.5)
    phylo_complete<-phylo_complete %<+% as.data.frame(states_traits)
    
    ASE_plot<-phylo_complete + geom_tiplab(offset = 0.25, size = 2.5, fontface = 4) + geom_tippoint(data=td_filter(isTip), aes(color=Cat), size=1) + scale_color_manual(values=c("#D2B48C", "#40826D"))

ggsave(ASE_plot, filename = "Ase_plot.pdf",  bg = "transparent", width = 10, height = 10)
}

In [ ]:
ASE_plot_maker(phy, best_fitting_model, states_traits)